In [1]:
import pickle
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pandas as pd
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import collections
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import DistilBertConfig

In [85]:
max_seq_length = 110
output_file = "data.tfrecord"

In [86]:
with open(".\orignal_data.pkl", "rb") as myprofile:  
    dt = pickle.load(myprofile)

In [87]:
tokenizer = DistilBertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = TFDistilBertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)
#.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFDistilBertForSequenceClassification: ['bert', 'nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the

In [2]:
class MyModel(tf.keras.Model):
    def __init__(self,model):
        super().__init__()
        #self.model = model.distilbert(input_ids,input_mask)[0]
        self.model = model
        self.model.trainable = False
        self.layersG = tf.keras.layers.GlobalMaxPool1D()
        self.layers1 = tf.keras.layers.Dense(50, activation="relu")
        self.layersD2 = tf.keras.layers.Dropout(0.2)
        self.layers3 = tf.keras.layers.Dense(10, activation="relu")
        self.layersD4 = tf.keras.layers.Dropout(0.2)
        self.layers5 = tf.keras.layers.Dense(2, activation="softmax")
        self.Bidirectional=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(60, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
    def call(self, inputs):
        input_ids = inputs["input_ids"]
        input_mask = inputs["input_mask"]
        embedding_layer  = self.model.distilbert(input_ids,input_mask)[0]
        X =self.Bidirectional(embedding_layer)
        X = self.layersG(X)
        X = self.layers1(X)
        X = self.layersD2(X)
        X = self.layers3(X)
        X = self.layersD4(X)
        X = self.layers5(X)
        return X

In [89]:
train_output_file = "train_data.tfrecord"
test_output_file = "test_data.tfrecord"

In [90]:
def select_data_and_label_from_record(record):
    x = {
        "input_ids": record["input_ids"],
        "input_mask": record["input_mask"],
        # 'segment_ids': record['segment_ids']
    }
    y = record["label_ids"]
    return (x, y)
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    return tf.io.parse_single_example(record, name_to_features)

In [91]:
def create_train_test_data(file_name,isTrain = False):
    dataset = tf.data.TFRecordDataset("data.tfrecord")
    if isTrain :
        dataset = dataset.repeat(500)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    name_to_features = {
            "input_ids": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "input_mask": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            # "segment_ids": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "label_ids": tf.io.FixedLenFeature([], tf.int64),
        }
    drop_remainder=False
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=100,
            drop_remainder=drop_remainder,
            num_parallel_calls=tf.data.experimental.AUTOTUNE,
        )
    )
    dataset.cache()
    re_dataset = dataset.map(select_data_and_label_from_record)
    return re_dataset

In [92]:
train_dataset = create_train_test_data(train_output_file,True)
test_dataset = create_train_test_data(test_output_file)

In [93]:
test_dataset

<MapDataset element_spec=({'input_ids': TensorSpec(shape=(None, 110), dtype=tf.int64, name=None), 'input_mask': TensorSpec(shape=(None, 110), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [94]:
learning_rate = 3e-3
epsilon = 1e-05

In [95]:
mode2 = MyModel(model)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
mode2.compile(optimizer=optimizer, loss=loss, metrics=[metric])
#mode2.summary()

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = mode2.fit(
    train_dataset,
    steps_per_epoch = 20,
    validation_data=test_dataset,
    validation_steps=5,
    #shuffle=True,
    epochs=50)

Epoch 1/50
20/20 [==============================] - 898s 45s/step - loss: 0.7138 - accuracy: 0.5285 - val_loss: 0.6715 - val_accuracy: 0.6280
Epoch 2/50
20/20 [==============================] - 891s 45s/step - loss: 0.6724 - accuracy: 0.6035 - val_loss: 0.6618 - val_accuracy: 0.6280
Epoch 3/50
20/20 [==============================] - 890s 45s/step - loss: 0.6429 - accuracy: 0.6150 - val_loss: 0.5741 - val_accuracy: 0.6280
Epoch 4/50
20/20 [==============================] - 884s 45s/step - loss: 0.5862 - accuracy: 0.6995 - val_loss: 0.5069 - val_accuracy: 0.8060
Epoch 5/50
20/20 [==============================] - 882s 45s/step - loss: 0.5216 - accuracy: 0.7555 - val_loss: 0.4575 - val_accuracy: 0.8060
Epoch 6/50
20/20 [==============================] - 880s 44s/step - loss: 0.5319 - accuracy: 0.7600 - val_loss: 0.4210 - val_accuracy: 0.8200
Epoch 7/50
20/20 [==============================] - 883s 45s/step - loss: 0.4910 - accuracy: 0.7860 - val_loss: 0.3994 - val_accuracy: 0.8620
Epoch 